![logo_ironhack_blue 7](https://user-images.githubusercontent.com/23629340/40541063-a07a0a8a-601a-11e8-91b5-2f13e4e6b441.png)

# Lab | Data cleaning and wrangling

For this lab, we will be using the same dataset we used in the previous labs. We recommend using the same notebook since you will be reusing the same variables you previous created and used in labs. 

### Instructions

So far we have worked on `EDA`. This lab will focus on data cleaning and wrangling from everything we noticed before.

1. We will start with removing outliers. So far, we have discussed different methods to remove outliers. Use the one you feel more comfortable with, define a function for that. Use the function to remove the outliers and apply it to the dataframe.
2. Create a copy of the dataframe for the data wrangling.
3. Normalize the continuous variables. You can use any one method you want.
4. Encode the categorical variables
5. The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful.
6. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

**Hint for Categorical Variables**

- You should deal with the categorical variables as shown below (for ordinal encoding, dummy code has been provided as well):

```python
# One hot to state
# Ordinal to coverage
# Ordinal to employmentstatus
# Ordinal to location code
# One hot to marital status
# One hot to policy type
# One hot to policy
# One hot to renew offercustomer_df
# One hot to sales channel
# One hot vehicle class
# Ordinal vehicle size

data["coverage"] = data["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})
# given that column "coverage" in the dataframe "data" has three categories:
# "basic", "extended", and "premium" and values are to be represented in the same order.
```




In [1]:
# Import libraries

import pandas as pd
import numpy as np
import scipy.stats as stats
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [2]:
data = pd.read_csv("files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv", sep=",")
data.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [3]:
data.columns = data.columns.str.lower().str.replace(' ', '_')

## 1. Removing outliers with a function

In [4]:
def remove_outliers(df, threshold=1.5, in_columns=data.select_dtypes(np.number).columns, skip_columns=[]):
    for column in in_columns:
        if column not in skip_columns:
            upper = np.percentile(data[column],75)
            lower = np.percentile(data[column],25)
            iqr = upper - lower
            upper_limit = upper + threshold * iqr
            lower_limit = lower - threshold * iqr
            df = df[(df[column]>lower_limit) & (df[column]<upper_limit)]
    return df

In [5]:
customer_df = remove_outliers(data, threshold=3, skip_columns=['number_of_open_complaints', 'total_claim_amount',]) 
customer_df.shape

(8700, 24)

## 2. Copying Dataset

In [6]:
data2 = customer_df.copy()

In [7]:
data2['effective_to_date'] = pd.to_datetime(data2['effective_to_date'])

## 3. Normalizing

In [8]:
continuous = data2.select_dtypes(include='float').columns.tolist()
scaler = MinMaxScaler()
data2[continuous] = scaler.fit_transform(data2[continuous])
data2[continuous].head()


,customer_lifetime_value,total_claim_amount
0,0.039493,0.165321
1,0.231869,0.486177
2,0.501445,0.243385
3,0.262273,0.227661
4,0.041782,0.059316


## 4. Encoding

In [9]:
#Ordinals first
ordinal = ['coverage', 'employmentstatus', 'location_code', 'vehicle_size']

data2['coverage'] = data2['coverage'].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})
data2['employmentstatus'] = data2['employmentstatus'].map({"Unemployed" : 0, "Other" : 1, "Employed" : 2})
data2['location_code'] = data2['location_code'].map({"Rural" : 0, "Suburban" : 1, "Urban" : 2})
data2['vehicle_size'] = data2['vehicle_size'].map({"Small" : 0, "Medsize" : 1, "Large" : 2})

In [10]:
# Get dummies 
data2 = pd.get_dummies(data2)
#Does not need to skip columns or select anything, get_dummies skip non-categorical columns by itself

## 5. Transforming Time Variable

In [11]:
data2['day'] = data2['effective_to_date'].dt.dayofweek
data2['month'] = data2['effective_to_date'].dt.month
data2 = data2.drop(columns=['effective_to_date'], axis=1)

## 6. Checking that everything is numerical

In [12]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8700 entries, 0 to 9133
Columns: 8757 entries, customer_lifetime_value to month
dtypes: float64(3), int64(11), uint8(8743)
memory usage: 73.5 MB


In [13]:
data2.head()

,customer_lifetime_value,coverage,employmentstatus,income,location_code,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,...,sales_channel_Call Center,sales_channel_Web,vehicle_class_Four-Door Car,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,day,month
0,0.039493,0,2.0,56274,1,69,32,5,0,1,...,0,0,0,0,0,0,0,1,3,2
1,0.231869,1,0.0,0,1,94,13,42,0,8,...,0,0,1,0,0,0,0,0,0,1
2,0.501445,2,2.0,48767,1,108,18,38,0,2,...,0,0,0,0,0,0,0,1,5,2
3,0.262273,0,0.0,0,1,106,18,65,0,7,...,1,0,0,0,0,1,0,0,3,1
4,0.041782,0,2.0,43836,0,73,12,44,0,1,...,0,0,1,0,0,0,0,0,3,2


## EXTRA

#### I followed the process but I think it would make more sense to normalize all the numericals except from the target (e.g: income)

In [14]:
data3 = customer_df.copy()

In [15]:
data3 = data3.drop(["income"],axis=1)

In [16]:
data3['effective_to_date'] = pd.to_datetime(data3['effective_to_date'])

In [17]:
continuous = data3.select_dtypes(include='number').columns.tolist()
scaler = MinMaxScaler()
data3[continuous] = scaler.fit_transform(data3[continuous])
data3[continuous].head()

,customer_lifetime_value,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,0.039493,0.047619,0.914286,0.050505,0.0,0.000,0.165321
1,0.231869,0.196429,0.371429,0.424242,0.0,0.875,0.486177
2,0.501445,0.279762,0.514286,0.383838,0.0,0.125,0.243385
3,0.262273,0.267857,0.514286,0.656566,0.0,0.750,0.227661
4,0.041782,0.071429,0.342857,0.444444,0.0,0.000,0.059316


In [18]:
#instead of giving them a number, we make them categoricals, and then we will encode them
data3['day'] = data3['effective_to_date'].dt.day_name().str.slice(stop=3)
data3['month'] = data3['effective_to_date'].dt.month_name().str.slice(stop=3)
data3 = data3.drop(columns=['effective_to_date'], axis=1)


In [19]:
data3 = pd.get_dummies(data3)

In [20]:
data3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8700 entries, 0 to 9133
Columns: 8773 entries, customer_lifetime_value to month_Jan
dtypes: float64(7), uint8(8766)
memory usage: 73.3 MB


In [21]:
data3.head()

,customer_lifetime_value,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount,customer_AA10041,customer_AA11235,customer_AA30683,...,vehicle_size_Small,day_Fri,day_Mon,day_Sat,day_Sun,day_Thu,day_Tue,day_Wed,month_Feb,month_Jan
0,0.039493,0.047619,0.914286,0.050505,0.0,0.000,0.165321,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,0.231869,0.196429,0.371429,0.424242,0.0,0.875,0.486177,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0.501445,0.279762,0.514286,0.383838,0.0,0.125,0.243385,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0.262273,0.267857,0.514286,0.656566,0.0,0.750,0.227661,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,0.041782,0.071429,0.342857,0.444444,0.0,0.000,0.059316,0,0,0,...,0,0,0,0,0,1,0,0,1,0


#### <div class="alert alert-block alert-info">I think it is more interesting like this.</div>

**Now we have everything in a scale from 0 to 1**